In [1]:
from util import *
from util_data_storage_and_load import *
import numpy as np
from numpy.linalg import inv
from scipy.sparse import csr_matrix, csc_matrix
import json

No dicts found; please check load_dicts...


In [2]:
with open('../temp_files/new_route_dict_journal.json', 'r') as json_file:
    new_route_dict = json.load(json_file)
    
number_of_routes = len(new_route_dict)

In [3]:
# implement GLS method to estimate OD demand matrix
def GLS(x, A, L):
    """
    x: sample matrix, each column is a link flow vector sample; number_of_links * K
    A: path-link incidence matrix
    P: logit route choice probability matrix
    L: dimension of xi
    ----------------
    return: xi
    ----------------
    """
    K = np.size(x, 1)
    S = samp_cov(x)

    #print("rank of S is: \n")
    #print(matrix_rank(S))
    #print("sizes of S are: \n")
    #print(np.size(S, 0))
    #print(np.size(S, 1))

    inv_S = inv(S).real

    A_t = A.transpose()

    Q_ = A_t * inv_S * A
    Q_ = Q_.real
#     Q = adj_PSD(Q_)  # Ensure Q to be PSD
    Q = Q_

    #print("rank of Q is: \n")
    #print(matrix_rank(Q))
    #print("sizes of Q are: \n")
    #print(np.size(Q, 0))
    #print(np.size(Q, 1))

    b = sum([A_t * inv_S * x[:, k] for k in range(K)])
    # print(b[0])
    # assert(1==2)

    model = Model("OD_matrix_estimation")

    xi = []
    for l in range(L):
        xi.append(model.addVar(name='xi_' + str(l)))

    model.update() 

    # Set objective: (K/2) xi' * Q * xi - b' * xi
    obj = 0
    for i in range(L):
        for j in range(L):
            obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
    for l in range(L):
        obj += - b[l] * xi[l]
    model.setObjective(obj)

    # Add constraint: xi >= 0
    for l in range(L):
        model.addConstr(xi[l] >= 0)
        #model.addConstr(xi[l] <= 5000)
    #fictitious_OD_list = zload('../temp_files/fictitious_OD_list')
    #for l in fictitious_OD_list:
        #model.addConstr(xi[l] == 0)
    model.update() 

    #model.setParam('OutputFlag', False)
    model.optimize()

    xi_list = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        xi_list.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return xi_list

In [4]:
# load link_route incidence matrix
A = zload('../temp_files/link_route_incidence_matrix_journal.pkz')

# load link counts data
with open('../temp_files/link_day_minute_Apr_dict_journal_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
# week_day_Apr_list = [9, 10, 11, 12, 13]

feasible_link_dict = zload('../temp_files/feasible_link_dict_journal.pkz')

link_day_minute_Apr_list = []
for link_idx in [feasible_link_dict[idx] for idx in range(len(feasible_link_dict))]:
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['PM_flow_minute'][minute_idx])

x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, len(feasible_link_dict), 2520)
# x = np.matrix.reshape(x, len(feasible_link_dict), 600)

# print(np.size(x,0), np.size(x,1))

x = np.nan_to_num(x)
# print(np.size(x,0), np.size(x,1))

# y = np.array(np.transpose(x))
# y = y[np.all(y != 0, axis=1)]
# x = np.transpose(y)
# x = np.matrix(x)

# print(np.size(x,0), np.size(x,1))
# print(x[:,:2])
# print(np.size(A,0), np.size(A,1))

### If we see an error message like:
#### GurobiError: Objective Q not PSD (diagonal adjustment of "some number" would be required), 
### where "some number" is very small, say, less than 1e-2, then, we can safely use the input flow data.

In [5]:
xi_list = GLS(x, A, number_of_routes)

Optimize a model with 1122 rows, 1122 columns and 1122 nonzeros
Model has 630003 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 2e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 1122 rows and 0 columns
Presolve time: 0.05s
Presolved: 0 rows, 1122 columns, 0 nonzeros
Presolved model has 630003 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 191
 AA' NZ     : 1.814e+04
 Factor NZ  : 1.834e+04 (roughly 1 MByte of memory)
 Factor Ops : 2.341e+06 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.62712676e+08 -2.33225515e+05  4.77e+04 3.10e+02  1.00e+06     0s
   1  -1.62753947e+07 -2.68210700e+07  1.12e+04 2.33e+02  2.86e+05     0s
   2   2.36528805e+07 -3.05209149e+07  3.78e+02 6.01e+00  5.67e+04     0s
   3   3.01463767e

In [6]:
xi_list

[63.77933695662713,
 471.8429920047723,
 9.77099310157579e-11,
 918.4884987756249,
 1.5657484208251866e-10,
 2.5863311996109564e-12,
 2.6935715757286695e-11,
 3.6016903997908543e-11,
 3.5764467107054246e-11,
 28.765821889569747,
 31.66633777460513,
 6.659469946102372e-11,
 29.69299777786242,
 8.60960633671622,
 2.192816875375869e-12,
 11.722161047214875,
 20.42819508270737,
 23.41448182749229,
 1.4432136679225053e-11,
 14.565209658579931,
 8.505551687610133,
 9.076266044216835,
 4.483896303685743,
 4.451529385767067,
 1.875185298100306,
 1.1605478064855677e-11,
 1.4195219191315993e-11,
 1.4198566042277278e-11,
 2.0233736806869933,
 1.8802037086510142,
 1.9366556712738237,
 1.9483561726411567,
 1.9658263937818887,
 44.582338748310015,
 118.68071848021494,
 381.4216989556132,
 5.1601597702858435e-12,
 167.19535123349885,
 1.5906972311914231e-12,
 104.26012576120355,
 90.96524652439908,
 3.051094654282264e-11,
 2.1172425277583727e-10,
 5.1147369232200454e-11,
 2.8978398476471806e-11,
 5.1